<a href="https://github.com/xuehangcang/DeepLearning/blob/main/docs/PyTorch/%E5%9F%BA%E7%A1%80%E7%9F%A5%E8%AF%86/5.构建神经网络.ipynb" download=""><svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="110" height="20" role="img" aria-label="jupyter: notebook"><title>jupyter: notebook</title><linearGradient id="s" x2="0" y2="100%"><stop offset="0" stop-color="#bbb" stop-opacity=".1"/><stop offset="1" stop-opacity=".1"/></linearGradient><clipPath id="r"><rect width="110" height="20" rx="3" fill="#fff"/></clipPath><g clip-path="url(#r)"><rect width="49" height="20" fill="#555"/><rect x="49" width="61" height="20" fill="#fe7d37"/><rect width="110" height="20" fill="url(#s)"/></g><g fill="#fff" text-anchor="middle" font-family="Verdana,Geneva,DejaVu Sans,sans-serif" text-rendering="geometricPrecision" font-size="110"><text aria-hidden="true" x="255" y="150" fill="#010101" fill-opacity=".3" transform="scale(.1)" textLength="390">jupyter</text><text x="255" y="140" transform="scale(.1)" fill="#fff" textLength="390">jupyter</text><text aria-hidden="true" x="785" y="150" fill="#010101" fill-opacity=".3" transform="scale(.1)" textLength="510">notebook</text><text x="785" y="140" transform="scale(.1)" fill="#fff" textLength="510">notebook</text></g></svg></a>

## 5.1 构建神经网络

神经网络由执行数据操作的层、模块组成。

`torch.nn` 命名空间提供了构建自己的神经网络所需的所有构建块。

PyTorch 中的每个模块都是 `nn.Module` 的子类。

神经网络本身就是一个模块，由其他模块（层）组成。这种嵌套结构使得构建和管理复杂的体系结构变得容易。

在接下来的章节中，我们将构建一个神经网络，用于对 FashionMNIST 数据集中的图像进行分类。

In [15]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 5.2 获取训练设备

我们希望能够在硬件加速器上，如GPU或MPS上训练我们的模型，如果可用的话。

让我们来检查一下 `torch.cuda` 或 `torch.backends.mps` 是否可用，否则我们就使用CPU。

In [16]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## 5.3 定义类

我们通过子类化 `nn.Module` 来定义神经网络，并在 `__init__` 中初始化神经网络层。

每个 `nn.Module` 子类都在 `forward` 方法中实现对输入数据的操作。

In [17]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

我们创建了一个 `NeuralNetwork` 实例，并将其移动到 `device` 中，并打印出其结构。

In [18]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


为了使用这个模型，我们需要将输入数据传递给它。这将执行模型的`前向传播`，同时进行一些`反向传播`。请勿直接调用 `model.forward()` !

将输入数据传递给模型后，会返回一个二维张量。

其中 dim=0 对应于每个类别的 10 个原始预测值的输出，而 dim=1 对应于每个输出的各个值。

我们可以通过将其通过 `nn.Softmax` 模块的实例来得到预测概率。

In [19]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


## 5.4 模型层

让我们来分解一下FashionMNIST模型中的层。

为了说明这一点，我们将取一个大小为 28x28 的样本小批量，其中包含 3 张图像，并观察当我们将它们通过网络传递时会发生什么。

In [20]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### 5.4.1 nn.Flatten

我们初始化 `nn.Flatten` 层，

将每个 2D 的 28 x 28 的图像转换为一个连续的包含 784 个像素值的数组。

In [21]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### 5.4.2 nn.Linear

`linear layer`是一个模块，它使用其存储的权重和偏置对输入进行线性变换。


In [22]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### 5.4.3 nn.ReLU

非线性激活函数是创建模型输入和输出之间复杂映射的关键。它们在线性变换之后应用，引入了非线性，帮助神经网络学习各种现象。

在这个模型中，我们在线性层之间使用 `nn.ReLU`，但是还有其他激活函数可以在您的模型中引入非线性。

In [23]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3549, -0.0539, -0.4223, -0.5090, -0.5241,  0.0671,  0.2640,  0.3148,
          0.4059, -0.0043,  0.2942, -0.3136, -0.4854, -0.4096,  0.6128, -0.0403,
          0.3008,  0.0272,  0.5043, -0.0982],
        [ 0.0611,  0.1303, -0.3039, -0.1856, -0.5054,  0.2207,  0.2165,  0.0329,
          0.2755, -0.1567,  0.2323, -0.2620, -0.5719, -0.1951,  0.4347, -0.3115,
          0.0223,  0.2554,  0.4202, -0.3879],
        [ 0.3159,  0.2250, -0.6701,  0.0093, -0.4254,  0.2786,  0.3091,  0.0265,
          0.3994,  0.0292,  0.4304, -0.3413, -0.1928, -0.2474,  0.5133,  0.1104,
         -0.0613,  0.2951,  0.5740, -0.3400]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3549, 0.0000, 0.0000, 0.0000, 0.0000, 0.0671, 0.2640, 0.3148, 0.4059,
         0.0000, 0.2942, 0.0000, 0.0000, 0.0000, 0.6128, 0.0000, 0.3008, 0.0272,
         0.5043, 0.0000],
        [0.0611, 0.1303, 0.0000, 0.0000, 0.0000, 0.2207, 0.2165, 0.0329, 0.2755,
         0.0000, 0.2323, 0.0000, 0.0000, 0.0000, 0.43

### 5.4.4 nn.Sequential

`nn.Sequential` 是一个有序的模块容器。数据会按照定义的顺序依次通过所有的模块。你可以使用序列容器来快速组合一个网络，比如 `seq_modules`。

In [24]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits

tensor([[-0.0407, -0.0436,  0.0431, -0.2591,  0.2633,  0.0719, -0.2602,  0.0352,
          0.2142,  0.2105],
        [-0.1104, -0.0551,  0.1166, -0.2316,  0.2608,  0.1021, -0.2817, -0.0521,
          0.3963,  0.2970],
        [-0.0170,  0.0762, -0.0094, -0.3774,  0.3341,  0.0778, -0.2287, -0.0870,
          0.3413,  0.3867]], grad_fn=<AddmmBackward0>)

### 5.4.5 nn.Softmax

神经网络的最后一层线性层返回 `logits`，这些值被传递到 `nn.Softmax` 模块中。这些 logits 被缩放到值为 [0, 1] 的概率表示，代表模型对每个类别的预测概率。`dim` 参数指定了哪个维度上的值必须总和为 1。

In [25]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.0924, 0.0921, 0.1005, 0.0743, 0.1253, 0.1034, 0.0742, 0.0997, 0.1192,
         0.1188],
        [0.0837, 0.0884, 0.1050, 0.0741, 0.1213, 0.1035, 0.0705, 0.0887, 0.1389,
         0.1258],
        [0.0910, 0.0999, 0.0917, 0.0634, 0.1292, 0.1000, 0.0736, 0.0848, 0.1302,
         0.1362]], grad_fn=<SoftmaxBackward0>)

## 5.5 模型参数

神经网络中的许多层都是*参数化*的，即具有相关的权重和偏置项，这些参数在训练过程中被优化。子类化 `nn.Module` 会自动跟踪模型对象中定义的所有字段，并使所有参数可以使用模型的 `parameters()` 或 `named_parameters()` 方法进行访问。

在此示例中，我们迭代每个参数，并打印其大小和值的预览。

In [26]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0209,  0.0339, -0.0216,  ...,  0.0168, -0.0327, -0.0261],
        [ 0.0091, -0.0300,  0.0073,  ...,  0.0261, -0.0312,  0.0122]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0057,  0.0282], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0081, -0.0040, -0.0374,  ...,  0.0388,  0.0413,  0.0277],
        [ 0.0179, -0.0427, -0.0428,  ..., -0.0084, -0.0159,  0.0429]],
       device='cuda:0', grad_fn=<Sl